In [1]:
import pygame
import numpy as np
import threading
import pyaudio
import scipy
import scipy.fftpack
import scipy.io.wavfile
import sys
import wave
from IPython.display import clear_output

pygame 2.5.2 (SDL 2.28.3, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
FORMAT = pyaudio.paInt16
CHANNELS = 1
SAMPLE_RATE = 44100
FFTSIZE = 512
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "file3.wav"
mic_recording = np.array([0]*FFTSIZE)
DEVICE = 1
W = 500

In [3]:
# import sounddevice as sd
# sd.query_devices()

In [3]:
def update_matrix(pcm):
    global screen_colors

    ffty = scipy.fftpack.fft(pcm)  # convert WAV to FFT
    ffty = abs(ffty[0:int(len(ffty)/2)])/500  # FFT is mirror-imaged
    # print("MIN:t%stMAX:t%s" % (min(ffty), max(ffty)))
    for i in range(len(ffty)):
        if ffty[i] < 0:
            ffty[i] = 0
        if ffty[i] > 255:
            ffty[i] = 255
    screen_colors = np.roll(screen_colors, -1, 0)
    screen_colors[-1] = ffty[::-1]

In [4]:
def record():
    global mic_recording
    p = pyaudio.PyAudio()
    Stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=SAMPLE_RATE,
                    frames_per_buffer=FFTSIZE,
                    input=True)
    while True:
        
        print(len(mic_recording))
        clear_output(wait = True)
        pcm = np.frombuffer( Stream.read(FFTSIZE) )
        mic_recording = np.append(mic_recording,pcm)

In [5]:
pal = [(max((x-128)*2, 0), x, min(x*2, 255)) for x in range(256)]

In [6]:
screen_colors = np.array(np.zeros((W, int(FFTSIZE/2))), dtype=int)

In [8]:
screen_colors.shape

(500, 256)

## Pygame settings

In [9]:
pygame.init()  # crank up PyGame
pygame.display.set_caption("Simple Spectrograph")
screen = pygame.display.set_mode((W, FFTSIZE/2))
world = pygame.Surface((W, FFTSIZE/2), depth=8)  # MAIN SURFACE
world.set_palette(pal)
clk = pygame.time.Clock()

t_rec = threading.Thread(target=record)  # make thread for record()
t_rec.daemon = True  # daemon mode forces thread to quit with program
t_rec.start()  # launch thread
while 1:
    for event in pygame.event.get():  # check if we need to exit
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    update_matrix(mic_recording[-FFTSIZE:])
    pygame.surfarray.blit_array(world, screen_colors)  # place data in window
    screen.blit(world, (0, 0))
    pygame.display.flip()  # RENDER WINDOW
    clk.tick(30)  # limit to 30FPS

SystemExit: 

C:\Users\chompu\.conda\envs\spectogram\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
len(mic_recording)

NameError: name 'mic_recording' is not defined

In [13]:

def graphFFT(pcm):
    global currentCol, data
    ffty = scipy.fftpack.fft(pcm)  # convert WAV to FFT
    ffty = abs(ffty[0:int(len(ffty)/2)])/500  # FFT is mirror-imaged
    # ffty=(scipy.log(ffty))*30-50 # if you want uniform data
    print("MIN:t%stMAX:t%s" % (min(ffty), max(ffty)))
    clear_output(wait = True)
    for i in range(len(ffty)):
        if ffty[i] < 0:
            ffty[i] = 0
        if ffty[i] > 255:
            ffty[i] = 255
    scooter.append(ffty)
    if len(scooter) < 6:
        return
    scooter.pop(0)
    ffty = (scooter[0]+scooter[1]*2+scooter[2]*3+scooter[3]*2+scooter[4])/9
    data = numpy.roll(data, -1, 0)
    data[-1] = ffty[::-1]
    currentCol += 1
    if currentCol == windowWidth:
        currentCol = 0


def record():
    p = pyaudio.PyAudio()
    inStream = p.open(format=pyaudio.paInt16, channels=1, rate=rate,
                      input_device_index=soundcard, input=True)
    linear = [0]*fftsize
    while True:
        linear = linear[int(fftsize/overlap):]
        pcm = numpy.frombuffer(inStream.read(
            int(fftsize/overlap)), dtype=numpy.int16)
        linear = numpy.append(linear, pcm)
        graphFFT(linear)


pal = [(max((x-128)*2, 0), x, min(x*2, 255)) for x in range(256)]
print( max(pal), min(pal))
data = numpy.array(numpy.zeros((windowWidth, int(fftsize/2))), dtype=int)
# data=Numeric.array(data) # for older PyGame that requires Numeric
pygame.init()  # crank up PyGame
pygame.display.set_caption("Simple Spectrograph")
screen = pygame.display.set_mode((windowWidth, fftsize/2))
world = pygame.Surface((windowWidth, fftsize/2), depth=8)  # MAIN SURFACE
world.set_palette(pal)
t_rec = threading.Thread(target=record)  # make thread for record()
t_rec.daemon = True  # daemon mode forces thread to quit with program
t_rec.start()  # launch thread
clk = pygame.time.Clock()
while 1:
    for event in pygame.event.get():  # check if we need to exit
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    pygame.surfarray.blit_array(world, data)  # place data in window
    screen.blit(world, (0, 0))
    pygame.display.flip()  # RENDER WINDOW
    clk.tick(30)  # limit to 30FPS

Exception in thread Thread-11 (record):
Traceback (most recent call last):
  File "C:\Users\chomps\.conda\envs\audios\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "C:\Users\chomps\.conda\envs\audios\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\chomps\AppData\Local\Temp\ipykernel_1308\3651637372.py", line 32, in record
  File "C:\Users\chomps\.conda\envs\audios\Lib\site-packages\pyaudio.py", line 608, in read
    return pa.read_stream(self._stream, num_frames, exception_on_overflow)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno -9981] Input overflowed


SystemExit: 

In [ ]:
soundcard = 1

In [ ]:
def record():
    p = pyaudio.PyAudio()
    inStream = p.open(format=pyaudio.paInt16, channels=1, rate=rate,
                      input_device_index=soundcard, input=True)
    linear = [0]*fftsize
    while True:
        linear = linear[fftsize/overlap:]
        pcm = numpy.fromstring(inStream.read(
            fftsize/overlap), dtype=numpy.int16)
        linear = numpy.append(linear, pcm)
        graphFFT(linear)

In [ ]:
record()

In [18]:
p = pyaudio.PyAudio()
inStream = p.open(format=pyaudio.paInt16, channels=1, rate=rate,
                      input_device_index=1, input=True)